In [ ]:
# Cosmic Rate from SRTrueParticles

In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#MC CV Sample --> High Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats.root"
infile = "../NTuples/mc_production_true_particles_ntuple_v2.root"

In [ ]:
inFile = uproot.open(infile)

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)

#slc_tree = inFile["slc_truth_tree"]
part_tree1 = inFile["part_tree1"]
#part_tree2 = inFile["part_tree2"]
part_tree3 = inFile["part_tree3"]


part_df1 = part_tree1.arrays(part_tree1.keys(), library="pd")
#part_df2 = part_tree2.arrays(part_tree2.keys(), library="pd")
part_df3 = part_tree3.arrays(part_tree3.keys(), library="pd")


part_df1[:2]

In [ ]:
#cosmic_df3["interaction_id"] = cosmic_df1["interaction_id"]

part_df3["pdg"] = part_df1["pdg"]
part_df3["interaction_id"] = part_df1["interaction_id"]

cosmic_df1 = part_df1.query("(pdg == 13 or pdg == -13) and interaction_id == -1")
#cosmic_df2 = part_df2.query("pdg == 13 or pdg == -13")
cosmic_df3 = part_df3.query("(pdg == 13 or pdg == -13) and interaction_id == -1")

cosmic_df3[:2]

In [ ]:
def is_AV(row):
    xs,ys,zs = row["start_x"], row["start_y"], row["start_z"]
    #xe,ye,ze = row["end_x"], row["end_y"], row["end_z"]
    v = -9998
    if (xs > v and ys > v and zs > v):
    #and xe > v and ye > v and ze > v):
        return 1
    else:
        return 0

cosmic_df3["AV"] = cosmic_df3.apply(is_AV, axis=1)
cosmic_df3[:4]

In [ ]:
def inTime(row):
    t = row["startT"]
    if 0 <= t <= 2:
        return 1
    else:
        return 0

cosmic_df3["inTime"] = cosmic_df3.apply(inTime, axis=1)
cosmic_df3[:2]

In [ ]:
#cosmic_tpc = cosmic_df3.query("AV == 1 and inTime == 1")
cosmic_tpc = cosmic_df3.query("AV == 1")
cosmic_tpc = cosmic_tpc.drop_duplicates()
cosmic_tpc[:2]

In [ ]:
t_cosmic = cosmic_tpc["startT"].values
delT = np.max(t_cosmic) - np.min(t_cosmic)
print(delT)

# Event Loop for Rate

In [ ]:
evt_tree = inFile["event_tree"]
evt_df = evt_tree.arrays(evt_tree.keys(), library="pd")
evt_df = evt_df.drop_duplicates()
evt_df[:2]

In [ ]:
runs = evt_df["run"].values
subruns = evt_df["subrun"].values

data = {'run': runs,
        'subrun': subruns,
       }

header_df = pd.DataFrame(data)
header_df = header_df.drop_duplicates()

header_df[:2]

In [ ]:
def get_genevt(row):
    r = row["run"]
    sr = row["subrun"]
    g = evt_df.query("run == "+str(r) + " and subrun == "+str(sr))["ngenevt"].values
    if g[-1] != g[0]:
        print("diff ?")
    return g[0]


header_df["genevt"] = header_df.apply(get_genevt, axis=1)
header_df[:3]

In [ ]:
N_cosmic_tot = 0
N_cosmic = 0

# Loop over events
for num in range(evt_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = evt_df.iloc[num]
    r, sr, e = row["run"], row["subrun"], row["evt"]

    # check cosmic number
    m1 = (cosmic_tpc["run"].values == r)
    m2 = (cosmic_tpc["subrun"].values == sr)
    m3 = (cosmic_tpc["evt"].values == e) 
    m = m1 & m2 & m3
    N = np.sum(m)
    N_cosmic_tot += N
    #N = len(np.array(cosmic_tpc["pdg"].values)[m])
    #print("number of cosmics", N_cosmic)
    if N > 0:
        N_cosmic += 1
    

print("Done!")

In [ ]:
print((1.0*N_cosmic)/evt_df.shape[0])

In [ ]:
E_tot = np.sum(header_df["genevt"].values)

print((1.0*N_cosmic)/E_tot)

In [ ]:
r = (1.0*N_cosmic_tot)/(E_tot*3000)

r *= 1.6

print(r)

In [ ]:
E_tot_p = header_df.shape[0]*50

In [ ]:
print(E_tot)
print(E_tot_p)

In [ ]:
num_runs = len(list(set(list(header_df["run"].values))))

E_tot_p2 = num_runs*50
print(E_tot_p2)

In [ ]:
r = (1.0*N_cosmic_tot)/(E_tot_p2*delT)

r *= 1.6

print(r)

In [ ]:
print(evt_df.shape[0]/E_tot)

In [ ]:
print(0.17571879170205024*0.14)

In [ ]:
print(0.17571879170205024*0.04)